In [ ]:
!git clone https://github.com/danishubin/sionna_dataset_generator.git

Cloning into 'sionna_dataset_generator'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 66 (delta 7), reused 7 (delta 6), pack-reused 54 (from 1)
Receiving objects: 100% (66/66), 30.74 MiB | 8.87 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
!pip install trimesh
!pip install rtree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install sionna

INFO: pip is looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.3/384.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.3 MB/s eta

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.15.1
Uninstalling tensorflow-2.15.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.11/dist-packages/tensorflow-2.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.15.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
   

In [ ]:
from tensorflow import keras

In [ ]:
import sionna

Adding Additional Trajectories

In [ ]:
import numpy as np

import trimesh

from math import tan, sin, cos, sqrt

import matplotlib.pyplot as plt

dt = 1.0
wheelbase = 0.5
cmds = [np.array([.8, np.random.choice([1, -1]) * .01])] * 20
step = 1

def move(x, dt, u, wheelbase):
    hdg = x[2]
    vel = u[0]
    steering_angle = u[1]
    dist = vel * dt
    if abs(steering_angle) > 0.001: # is robot turning?
        beta = (dist / wheelbase) * tan(steering_angle)
        r = wheelbase / tan(steering_angle) # radius
        sinh, sinhb = sin(hdg), sin(hdg + beta)
        cosh, coshb = cos(hdg), cos(hdg + beta)
        return x + np.array([-r*sinh + r*sinhb,
                              r*cosh - r*coshb, beta]), np.array([-vel*sinh + vel*sinhb, vel*cosh - vel*coshb]), hdg
    else: # moving in straight line
        return x + np.array([dist*cos(hdg), dist*sin(hdg), 0]), np.array([vel*cos(hdg), vel*sin(hdg)]), hdg

if __name__ == "__main__":

    trajectories = np.empty((0,20,3))

    for i in range(20):
        cmds = [np.array([np.random.choice([1, -1]) * np.random.uniform(.2, .8), np.random.choice([1, -1]) * .01])] * 20
        # Generate trajectories
        track = []
        velocity = []
        heading = []
        sim_pos = np.array([np.random.uniform(10, 50), np.random.uniform(-5, 2), np.random.uniform(0, np.pi / 2)])
        # sim_pos = np.array([30, 5, 0])
        for i, u in enumerate(cmds):
            sim_pos, vel, hdg = move(sim_pos, dt/step, u, wheelbase)
            track_tmp = np.copy(sim_pos)
            track_tmp[2] = -10
            track.append(track_tmp)
            velocity.append(vel)
            heading.append(hdg)
        track = np.array(track)
        velocity = np.array(velocity)
        heading = np.array(heading)

        # plt.plot(track[:, 0], track[:,1], marker='.', color='k', lw=2)
        # plt.axis('equal')
        # plt.title("Robot  Trajectory")
        # plt.show()

        mesh = trimesh.load_mesh("sionna_dataset_generator/models/canyon.ply")

        # create some rays
        ray_origins = track #np.array([[0, 0, -5], [2, 2, -10]])
        ray_directions = np.array([[0, 0, 1]]*track.shape[0])

        # run the mesh- ray test
        locations, index_ray, index_tri = mesh.ray.intersects_location(
            ray_origins=ray_origins, ray_directions=ray_directions
        )
        # import pdb; pdb.set_trace()
        if len(locations) < len(cmds) or locations.shape != (20, 3):
            continue
        # stack rays into line segments for visualization as Path3D
        ray_visualize = trimesh.load_path(
            np.hstack((ray_origins[:1], ray_origins[:1] + ray_directions[:1])).reshape(-1, 2, 3)
        )
        # if locations.shape != (20, 3):
        #     import pdb; pdb.set_trace()

        locations = locations[np.argsort(index_ray)]
        locations[:, 2] += 0.5
        locations_reshaped = locations.reshape(1, 20, 3)
        trajectories = np.concatenate((trajectories, locations_reshaped), axis=0)

        print(trajectories.shape)

        # create a visualization scene with rays, hits, and mesh
        scene = trimesh.Scene([mesh, ray_visualize, trimesh.points.PointCloud(locations)])

        scene.show()
        # # mesh.show()

    np.save("trajectories.npy", trajectories)

(1, 20, 3)
(2, 20, 3)
(3, 20, 3)
(4, 20, 3)
(5, 20, 3)
(6, 20, 3)
(7, 20, 3)
(8, 20, 3)
(9, 20, 3)
(10, 20, 3)
(11, 20, 3)
(12, 20, 3)
(13, 20, 3)
(14, 20, 3)
(15, 20, 3)
(16, 20, 3)
(17, 20, 3)
(18, 20, 3)
(19, 20, 3)
(20, 20, 3)
(21, 20, 3)
(22, 20, 3)
(23, 20, 3)
(24, 20, 3)
(25, 20, 3)
(26, 20, 3)
(27, 20, 3)
(28, 20, 3)
(29, 20, 3)
(30, 20, 3)
(31, 20, 3)
(32, 20, 3)
(33, 20, 3)
(34, 20, 3)
(35, 20, 3)
(36, 20, 3)
(37, 20, 3)
(38, 20, 3)
(39, 20, 3)
(40, 20, 3)
(41, 20, 3)
(42, 20, 3)
(43, 20, 3)
(44, 20, 3)
(45, 20, 3)
(46, 20, 3)
(47, 20, 3)
(48, 20, 3)
(49, 20, 3)
(50, 20, 3)
(51, 20, 3)
(52, 20, 3)
(53, 20, 3)
(54, 20, 3)
(55, 20, 3)
(56, 20, 3)
(57, 20, 3)


Creating Dataset

In [ ]:
# Jason's implementation of directional algo
from algorithim import Params, Algorithm, Capon
from scipy.signal import find_peaks

import os
os.environ['DRJIT_LIBLLVM_PATH'] = '/usr/lib/x86_64-linux-gnu/libLLVM.so:'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import os # Configure which GPU

if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
    print(f"Using GPU {gpu_num}")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.list_physical_devices('GPU')
print(f"Found {len(gpus)} GPUs")
if gpus:
    print(f"Found GPU: {gpus[0].name}")
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    no_preview = True # deactivate preview
except:
    if os.getenv("SIONNA_NO_PREVIEW"):
        no_preview = True
    else:
        no_preview = False

resolution = [480,320] # increase for higher quality of renderings

# Define magic cell command to skip a cell if needed
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

# Set random seed for reproducibility
sionna.config.seed = 42



[]
Found 0 GPUs


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, AntennaArray, Antenna

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement
from sionna.ofdm import ResourceGrid

def calculate_heading_rad(start, end):
    # Calculate the difference vector
    diff = np.array(end) - np.array(start)

    # Calculate the heading angle in radians
    heading = np.arctan2(diff[1], diff[0])

    return heading

def get_rss_from_csi(csi):
  # Compute CSI power (power per subcarrier)
  csi_power = tf.abs(csi)**2

  # Compute RSS per receiver-transmitter pair (sum over subcarriers)
  rss_per_rx_tx = tf.reduce_sum(csi_power, axis=-1)  # Sum over subcarriers

  # Convert RSS from linear scale (e.g., Watts) to dBm
  rss_per_rx_tx_dBm = 10 * tf.math.log(rss_per_rx_tx / 1e-3) / tf.math.log(10.0)

  return rss_per_rx_tx_dBm

def dbm_to_watts(dbm):
    return 10. ** ((dbm-30)/10)

def watts_to_dbm(watts):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return 10 * np.log10(watts + epsilon) + 30

# # Multiple Antennas like ASUS Router
def get_antenna_positions(spacing):
    return np.array([[0.0, spacing / 2 + spacing , 0.0],
                     [0.0, spacing / 2, 0.0],
                     [0.0, -spacing / 2, 0.0],
                     [0.0, -spacing / 2 - spacing, 0.0]])

def flip_trajectory(data):
    # Vector from origin to the first and last points of each trajectory
    first_points = data[:, 0, :]  # Shape (1000, 3)
    last_points = data[:, -1, :]  # Shape (1000, 3)

    # Compute the dot product of the first and last points with respect to the origin
    dot_products = np.sum(first_points * last_points, axis=1)

    # Identify trajectories heading away from the origin (dot product > 0)
    away_from_origin = dot_products < 0

    # Flip the trajectories that are not heading away from the origin
    data[~away_from_origin] = data[~away_from_origin, ::-1, :]

    return data

In [ ]:
# List for dataset
angles = []
angle_profile_values = []
rssi = []
rssi_normalized = []
data = []
total_traj = 20 # len(trajectories)

for traj in range(total_traj):
    SCENE_NAME = "canyon"
    IMAGE_FOLDER = f"images/{SCENE_NAME}"

    # Load integrated scene
    scene = load_scene(f"sionna_dataset_generator/models/{SCENE_NAME}.xml") # Try also sionna.rt.scene.etoile

    # Configure antenna array for all transmitters
    scene.tx_array = PlanarArray(num_rows=1,
                                num_cols=1,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="iso",
                                polarization="V")

    antenna_positions = get_antenna_positions(0.03)
    antenna_array_angle = -175 # Default

    RelativeAntennas = AntennaArray(antenna=Antenna("dipole", "V"),
                                positions=tf.Variable(antenna_positions.tolist()))

    # scene.tx_array = RelativeAntennas
    scene.rx_array = RelativeAntennas

    # Create transmitter
    tx = Transmitter(name="tx",
                    position=[0., -10., 0.],
                    orientation=[np.radians(0),0,0])

    # Add transmitter instance to scene
    scene.add(tx)

    trajectories = np.load('sionna_dataset_generator/trajectories_2.npy')
    trajectories = flip_trajectory(trajectories)
    trajectory_index = traj
    num_steps = trajectories.shape[1]
    trajectory_steps =  np.arange(num_steps - 1)

    start_color = np.array([1, 0, 0])  # Red
    end_color = np.array([0, 0, 1])
    colors = [start_color + (end_color - start_color) * i / (num_steps - 1) for i in range(num_steps)]

    # Calculate headings for the trajectory
    calculated_headings = []
    for i in trajectory_steps: #range(0, len(trajectories[0]) - 1):
        heading = calculate_heading_rad(trajectories[trajectory_index,i,:2], trajectories[trajectory_index,i + 1,:2])
        calculated_headings.append(heading)

    rxs = []
    for i, position in enumerate(trajectories[trajectory_index, np.arange(num_steps) ,:]):
        if i == (trajectories[0].shape[0] - 1):
            heading = calculated_headings[-1]
        else:
            heading = calculated_headings[i]
        rxs.append(Receiver(name=f"rx{i}",
                position=position,
                orientation=[heading,0,0]))
        scene.add(rxs[i])

    antenna_array_angle = np.radians(180) # heading

    scene.frequency = 2.14e9 #5.745 # in Hz; implicitly updates RadioMaterials

    scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

    # Compute propagation paths
    paths = scene.compute_paths(max_depth=5,
                                num_samples=1e6,  # Number of rays shot into directions defined
                                                # by a Fibonacci sphere , too few rays can
                                                # lead to missing paths
                                los=True,  # Include Line-of-Sight paths
                                reflection=True,  # Include reflection paths
                                diffraction=True,  # Include diffraction paths
                                scattering=True,  # Include scattering paths
                                )

    paths.normalize_delays = False

    # Determine subcarrier frequencies
    rg = ResourceGrid(num_ofdm_symbols=1,
                    fft_size=52,
                    dc_null = True,
                    cyclic_prefix_length=20,
                    #   pilot_pattern = "kronecker",
                    #   pilot_ofdm_symbol_indices = [2, 8],
                    subcarrier_spacing=5e6) #30e3)

    frequencies = subcarrier_frequencies(rg.fft_size, rg.subcarrier_spacing)

    # get rss
    a, tau = paths.cir()

    csi = cir_to_ofdm_channel(frequencies, a, tau, normalize=False)  # Non-normalized includes path-loss

    csi_reshaped = []
    for i in range(len(rxs)):
        csi_reshaped.append(np.array(csi[:,i,:,:,:,:,:]).reshape(4, 52))

    #Output: h_f ([batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_steps, fft_size], tf.complex) – Channel frequency responses at frequencies
    block_rss = get_rss_from_csi(csi).numpy()[0, :, 0, 0, 0, 0]
    block_rss[np.isneginf(block_rss)] = -90
    rssi.append(block_rss[:-1])
    # Define range
    min_rss = -90
    max_rss = 0

    # Normalize values to 0-1
    normalized = (block_rss - min_rss) / (max_rss - min_rss)
    rssi_normalized.append(normalized[:-1])

    # Define three colors
    color_black = np.array([0, 0, 0])
    color_red = np.array([1, 0, 0])
    color_blue = np.array([0, 0, 1])
    colors = []
    for value in normalized:
        if value <= 0.5:
            # Scale between black and gray
            color = color_black + (color_red - color_black) * (value / 0.5)
        else:
            # Scale between gray and white
            color = color_red + (color_blue - color_red) * ((value - 0.5) / 0.5)
        colors.append(color)

    # Map normalized values to colors
    # 0-1 range is split into 3 intervals: [0, 1/3), [1/3, 2/3), [2/3, 1]
    # color_indices = np.digitize(normalized, bins=[1/3, 2/3], right=False)
    # mapped_colors = [colors[idx] for idx in color_indices]

    for i, rx in enumerate(rxs):
        rx.color = colors[i]


    # Implementing Capon Algorithim
    params = Params(antenna_positions[:, :2])

    angle_tmp = []
    angle_profile_values_tmp = []
    for i in range(0, len(rxs) - 1):

        csi_tmp = tf.reshape(csi[:,i,:,:,:,:,:], [4, 52]) #  tf.reshape(csi[:,i,:,:,:,:,:], [1, -1])

        AOA, theta_samples, profile_norm = Capon(params, 155, 80, csi_tmp).evaluate()

        # This for when going towards transmitter
        profile_norm = np.roll(profile_norm, 180)
        # Capture three top peaks from bottom of profile (90-270 degrees)
        evaluation = np.array([profile_norm[90:270]])
        evaluation = evaluation.flatten()
        peaks, _ = find_peaks(evaluation)
        peak_values = evaluation[peaks]
        angle_profile_values_tmp.append(np.pad(peak_values, (0, max(0, 3 - peak_values.shape[0])), mode='constant'))
        angle_tmp.append(np.pad(theta_samples[90:270][peaks], (0, max(0, 3 - theta_samples[90:270][peaks].shape[0])), mode='constant'))
        AOA = np.array([AOA])


    angles.append(angle_tmp)
    angle_profile_values.append(angle_profile_values_tmp)

for i in range(0, len(rssi)):
  prev_angles = []
  prev_angle_profiles = []
  prev_rssi = []
  for j in range(0, len(rssi[i]) - 1):
    prev_angles.append([angles[i][j][0], angles[i][j][1], angles[i][j][2]])
    prev_angle_profiles.append([angle_profile_values[i][j][0], angle_profile_values[i][j][1], angle_profile_values[i][j][2]])
    prev_rssi.append(rssi_normalized[i][j])
  data.append([rssi_normalized[i][len(rssi[i]) - 1], prev_angles, prev_angle_profiles, prev_rssi])

In [ ]:
print(data)
print(len(data))
print(len(data[0][1]))

NameError: name 'data' is not defined

In [ ]:
print(len(rssi))
print(len(rssi[0]))

20
19


In [ ]:
import csv

def list_to_csv(data_list, filename):
    """
    Writes a list of lists to a CSV file.

    Args:
        data_list (list of lists): The data to write to the CSV file.
        filename (str): The name of the CSV file to create.
    """
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data_list)

list_to_csv(data, 'dataset.csv')